In [ ]:
from primeqa.components.retriever.dense import ColBERTRetriever
from primeqa.pipelines.qa_pipeline import QAPipeline
import json

index_root = "/dccstor/mabornea1/kilt-wikipedia-test/colbert_ir/kilt_wikipedia_eli5_dev_exp/indexes/"
index_name = "kilt_wikipedia_eli5_dev_indname"
collection = "/dccstor/mabornea1/kilt-wikipedia-test/passages/kilt_knowledgesource_eli5_dev.tsv"


retriever = ColBERTRetriever(index_root = index_root, 
                                     index_name = index_name, 
                                     collection = collection, 
                                     max_num_documents = 3)
retriever.load()

In [ ]:
from primeqa.components.reader.prompt import PromptGPTReader

reader = PromptGPTReader(api_key="")
reader.load()

pipeline = QAPipeline(retriever, reader)

In [ ]:
questions = ["What causes the trail behind jets at high altitude?"]
answers = pipeline.run(questions)
print(json.dumps(answers, indent=4))

In [ ]:
from primeqa.components.reader.prompt import PromptFLANReader

reader = PromptFLANReader()
reader.load(model="flan-t5-xxl", use_bam=False)

#pipeline = QAPipeline(retriever, reader)

In [1]:
from primeqa.components.reader.prompt import PromptFLANReader
from primeqa.pipelines.qa_pipeline import QAPipeline

bamreader = PromptFLANReader(api_key="pak-EsKayQ0iw6gj8Bw8JYbg3G3Ye_iMIqwop9aJlzRoz40")
bamreader.load(model="flan-t5-xxl", use_bam=True)


In [2]:
import json
questions = ["Answer the question: What causes the trail behind jets at high altitude?"]
answers = bamreader.predict(questions, contexts=[""])
print(json.dumps(answers, indent=4))

/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bam-api.res.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[
    {
        "example_id": 0,
        "text": "condensation of water droplets on wings and fuselage of aircrafts due to high temperature and low humidity in the air at high altitudes resulting in a trail of water droplets behind the aircrafts. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . "
    }
]
